In [ ]:
!pip install transformers torch lime scikit-learn pandas tqdm

In [31]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import re
import nltk
from nltk.corpus import stopwords
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from lime.lime_text import LimeTextExplainer
from tqdm import tqdm

# Download NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Fix for AdamW Import across different library versions
try:
    from transformers import AdamW
except ImportError:
    from torch.optim import AdamW

# 1. ENHANCED DATA CLEANING
def clean_news_text(text):
    # Remove Reuters headers and common noise
    text = re.sub(r'^.*?\(Reuters\)\s-\s', '', str(text))
    text = re.sub(r'[^\w\s]', '', text).lower()
    words = text.split()
    # Filter stopwords so LIME highlights meaningful "Fake News" markers
    cleaned = [w for w in words if w not in stop_words]
    return " ".join(cleaned)

# 2. DATA PREPARATION (Scaling to 5,000 for better stability)
try:
    true_df = pd.read_csv("/content/drive/MyDrive/Machine_test_camp6/dataset/True.csv")
    fake_df = pd.read_csv("/content/drive/MyDrive/Machine_test_camp6/dataset/Fake.csv")
    true_df['label'], fake_df['label'] = 1, 0
    df = pd.concat([true_df, fake_df]).sample(frac=1).reset_index(drop=True).head(5000)
    df['cleaned_text'] = df['text'].apply(clean_news_text)
except Exception as e:
    print(f"File Error: {e}. Ensure True.csv and Fake.csv are in the folder.")

# 3. TOKENIZATION
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
X_train, X_val, y_train, y_val = train_test_split(df['cleaned_text'], df['label'], test_size=0.2)

def get_tokens(texts, labels):
    encoded = tokenizer(list(texts), truncation=True, padding=True, max_length=128, return_tensors='pt')
    return TensorDataset(encoded['input_ids'], encoded['attention_mask'], torch.tensor(list(labels)))

train_ds = get_tokens(X_train, y_train)
val_ds = get_tokens(X_val, y_val)

# 4. MODEL & STABILIZED HYPERPARAMETERS
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8) # Stabilized LR

# Linear Warmup helps BERT not 'jitter' on small datasets
epochs = 3
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.1 * total_steps), num_training_steps=total_steps)

# 5. TRAINING LOOP
print(f"Training on {device}...")
model.train()
for epoch in range(epochs):
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for batch in loop:
        optimizer.zero_grad()
        input_ids, mask, labels = [t.to(device) for t in batch]
        outputs = model(input_ids, attention_mask=mask, labels=labels)
        outputs.loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Prevents gradient explosion
        optimizer.step()
        scheduler.step()
        loop.set_postfix(loss=outputs.loss.item())



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: bert-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.bias                       | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
classifier.bias                            | MISSING    | 
classifier.weight                          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Training on cuda...


Epoch 3: 100%|██████████| 250/250 [01:47<00:00,  2.33it/s, loss=0.000811]



--- METRICS ---
Accuracy: 0.9940
F1-Score: 0.9937

INPUT: URGENT: Secret military sources confirm that the moon landing was filmed in a ba...
RESULT: FAKE (82.97%)
SUSPICIOUS KEYWORDS: ['secret', 'basement', 'urgent']
EXPLANATION: This article shows linguistic alignment with fake news patterns, heavily influenced by terms like: secret, basement, urgent.


In [32]:
# 6. EVALUATION METRICS
model.eval()
val_loader = DataLoader(val_ds, batch_size=16)
preds, actuals = [], []
with torch.no_grad():
    for batch in val_loader:
        input_ids, mask, labels = [t.to(device) for t in batch]
        out = model(input_ids, attention_mask=mask)
        preds.extend(torch.argmax(out.logits, dim=1).cpu().numpy())
        actuals.extend(labels.cpu().numpy())

p, r, f1, _ = precision_recall_fscore_support(actuals, preds, average='binary')
print(f"\n--- METRICS ---\nAccuracy: {accuracy_score(actuals, preds):.4f}\nF1-Score: {f1:.4f}")

# 7. EXPLAINABLE AI OUTPUT
def predict_and_explain(text):
    cleaned = clean_news_text(text)
    def predictor(texts):
        toks = tokenizer(texts, return_tensors='pt', truncation=True, padding=True, max_length=128).to(device)
        with torch.no_grad():
            logits = model(**toks).logits
            return F.softmax(logits, dim=1).cpu().numpy()

    probs = predictor([cleaned])[0]
    label = "REAL" if np.argmax(probs) == 1 else "FAKE"

    explainer = LimeTextExplainer(class_names=['FAKE', 'REAL'])
    exp = explainer.explain_instance(cleaned, predictor, num_features=8)
    # Filter words that actually influenced the specific prediction
    keywords = [w for w, weight in exp.as_list() if (weight < 0 and label == "FAKE") or (weight > 0 and label == "REAL")]

    print("\n" + "="*40)
    print(f"INPUT: {text[:80]}...")
    print(f"RESULT: {label} ({probs.max():.2%})")
    print(f"SUSPICIOUS KEYWORDS: {keywords[:5]}")
    print(f"EXPLANATION: This article shows linguistic alignment with {label.lower()} news patterns, heavily influenced by terms like: {', '.join(keywords[:3])}.")
    print("="*40)

# 8. THE FINAL TEST
predict_and_explain("The Federal Reserve announced on Wednesday that it will maintain current interest rates following its two-day policy meeting. Officials stated that the decision aims to balance inflation targets with sustainable economic growth, according to the formal report released in Washington.")


--- METRICS ---
Accuracy: 0.9940
F1-Score: 0.9937

INPUT: The Federal Reserve announced on Wednesday that it will maintain current interes...
RESULT: REAL (99.97%)
SUSPICIOUS KEYWORDS: ['wednesday', 'announced', 'targets', 'reserve', 'released']
EXPLANATION: This article shows linguistic alignment with real news patterns, heavily influenced by terms like: wednesday, announced, targets.
